In [ ]:
import json 
import uuid 
 
class Book: 
    def __init__(self, title, author, year, status="в наличии"): 
        self.id = str(uuid.uuid4())
        self.title = title 
        self.author = author 
        self.year = year 
        self.status = status 
 
    def to_dict(self): 

        return { 
            "id": self.id, 
            "title": self.title, 
            "author": self.author, 
            "year": self.year, 
            "status": self.status 
        } 
 
    @staticmethod 
    def from_dict(data): 
        book = Book(data["title"], data["author"], data["year"], data["status"]) 
        book.id = data["id"] 
        return book 
 
 
class Library: 
    def __init__(self, storage_file="library.json"): 
        self.storage_file = storage_file 
        self.books = self.load_books() 
 
    def load_books(self): 
        """ 
        Загружает книги из файла JSON, если он существует. 
        """ 
        try: 
            with open(self.storage_file, "r", encoding="utf-8") as file: 
                data = json.load(file) 
                return [Book.from_dict(book) for book in data] 
        except FileNotFoundError: 
            return [] 
        except json.JSONDecodeError: 
            print("Ошибка чтения файла. Создаётся новая библиотека.") 
            return [] 
 
    def save_books(self): 
        """ 
        Сохраняет книги в файл JSON. 
        """ 
        with open(self.storage_file, "w", encoding="utf-8") as file: 
            json.dump([book.to_dict() for book in self.books], file, ensure_ascii=False, indent=4) 
 
    def add_book(self, title, author, year): 
        """ 
        Добавляет новую книгу в библиотеку. 
        """ 
        new_book = Book(title, author, year) 
        self.books.append(new_book) 
        self.save_books() 
        return new_book 
 
    def remove_book(self, book_id): 
        """ 
        Удаляет книгу по её ID. 
        """ 
        for book in self.books: 
            if book.id == book_id: 
                self.books.remove(book) 
                self.save_books() 
                return True 
        return False 
 
    def find_books(self, query, field): 
        """ 
        Ищет книги по указанному полю (title, author, year). 
        """ 
        return [book for book in self.books if str(getattr(book, field)).lower() == str(query).lower()] 
 
    def change_status(self, book_id, new_status): 
        """ 
        Изменяет статус книги по ID. 
        """ 
        for book in self.books: 
            if book.id == book_id: 
                book.status = new_status 
                self.save_books() 
                return True 
        return False 
 
    def list_books(self): 
        """ 
        Возвращает список всех книг в библиотеке. 
        """ 
        return self.books 
 
 
def library_cli(): 
    library = Library()
 
    print("Добро пожаловать в библиотеку! Введите команду:") 
    print("Команды: add, remove, find, list, status, exit") 
 
    while True: 
        command = input("\n> ").strip().lower() 
 
        if command == "add": 
            title = input("Введите название книги: ") 
            author = input("Введите автора книги: ") 
            year = input("Введите год издания книги: ") 
 
            if not year.isdigit(): 
                print("Ошибка: год издания должен быть числом.") 
                continue 
 
            new_book = library.add_book(title, author, int(year)) 
            print(f"Книга добавлена: {new_book.id} - {new_book.title}") 
 
        elif command == "remove": 
            book_id = input("Введите ID книги для удаления: ") 
            if library.remove_book(book_id): 
                print("Книга успешно удалена.") 
            else: 
                print("Ошибка: книга с таким ID не найдена.") 
 
        elif command == "find": 
            field = input("Введите поле для поиска (title, author, year): ").lower() 
            if field not in {"title", "author", "year"}: 
                print("Ошибка: поле поиска должно быть title, author или year.") 
                continue 
 
            query = input(f"Введите значение для поиска по {field}: ") 
            results = library.find_books(query, field) 
            if results: 
                print("Найденные книги:") 
                for book in results: 
                    print(f"{book.id} - {book.title} by {book.author}, {book.year}, статус: {book.status}") 
            else: 
                print("Книги не найдены.") 
 
        elif command == "list": 
            books = library.list_books() 
            if books: 
                print("Список всех книг:") 
                for book in books: 
                    print(f"{book.id} - {book.title} by {book.author}, {book.year}, статус: {book.status}") 
            else: 
                print("Библиотека пуста.") 
 
        elif command == "status": 
            book_id = input("Введите ID книги: ") 
            new_status = input("Введите новый статус (в наличии, выдана): ").strip() 
 
            if new_status not in {"в наличии", "выдана"}: 
                print("Ошибка: статус должен быть 'в наличии' или 'выдана'.") 
                continue 
 
            if library.change_status(book_id, new_status): 
                print("Статус книги успешно изменен.") 
            else: 
                print("Ошибка: книга с таким ID не найдена.") 
 
        elif command == "exit": 
            print("Выход из программы. До свидания!") 
            break 
 
        else: 
            print("Неизвестная команда. Попробуйте снова.") 
 
 

library_cli()
